![Problem Statement](https://images.aicrowd.com/uploads/ckeditor/pictures/711/content_embedding_problem_statement.png)



<p align="center"> Join the communty! <br><a href="https://discord.gg/GTckBMx"><img src="https://img.shields.io/discord/657211973435392011?style=for-the-badge" alt="chat on Discord"></a>
</p>

# How to use this notebook 📝

1. **Copy the notebook**. This is a shared template and any edits you make here will not be saved. _You should copy it into your own drive folder._ For this, click the "File" menu (top-left), then "Save a Copy in Drive". You can edit your copy however you like.
2. **Link it to your AIcrowd account**. In order to submit your predictions to AIcrowd, you need to provide your account's API key.



# Setup AIcrowd Utilities 🛠

In [2]:
!pip install -U aicrowd-cli
%load_ext aicrowd.magic

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 1.1 MB 2.8 MB/s eta 0:00:01
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 22.3.0
    Uninstalling pyzmq-22.3.0:
      Successfully uninstalled pyzmq-22.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-client 7.2.2 requires pyzmq>=22.3, but you have pyzmq 22.1.0 which is incompatible.



## Login to AIcrowd 


In [3]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/pPIh-Gk5QCoEXF8r-gLXDwAibCFPGdLVjvcSWC8WZfU
API Key valid
Gitlab access token valid
Saved details successfully!


# Install packages 🗃

Please add all pacakages installations in this section

In [3]:
!pip install torch torchvision simclr

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# Import necessary modules and packages 📚


In [5]:
import os
import cv2
import numpy as np
from tqdm.auto import tqdm

import torch
import torchvision
import torchvision.transforms as T

from simclr import SimCLR
from simclr.modules import NT_Xent
from simclr.modules import LARS

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_19448/1641836207.py:10 in <cell line: 10>                                  │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_19448/1641836207.py'                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'SimCLR' from 'simclr' (unknown location)

# Download and prepare the dataset 🔍

In [6]:
aicrowd_challenge_name = "mabe-2022-ant-beetles-video-data"
if not os.path.exists('data'):
    os.mkdir('data')

datafolder = 'data/'

## If data is already downloaded and stored on google drive, skip the download and point to the prepared directory
# datafolder = '/content/drive/MyDrive/mabe-2022-ant-beetle/data/'

video_folder = f'{datafolder}video_clips/'
print(video_folder)

data/video_clips/


In [ ]:
## The download might take a while, recommend to move to Google Drive if you want to run multiple times.
%aicrowd ds dl -c {aicrowd_challenge_name} -o data *.npy* # Download all files
# We'll download the 224x224 videos since they're fast on the dataloader, but you can use the full sized videos if you want
%aicrowd ds dl -c {aicrowd_challenge_name} -o data *resized_224* # Download all file
# %aicrowd ds dl -c {aicrowd_challenge_name} -o data *videos.zip* # Download the 512x512 videos

frame_number_map.npy:   0%|          | 0.00/436k [00:00<?, ?B/s]

sample_submission.npy:   0%|          | 0.00/171M [00:00<?, ?B/s]

submission_keypoints.npy:   0%|          | 0.00/138M [00:00<?, ?B/s]

user_train.npy:   0%|          | 0.00/49.4M [00:00<?, ?B/s]

submission_videos_resized_224.zip:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

userTrain_videos_resized_224.zip:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

In [7]:
%aicrowd ds dl -c {aicrowd_challenge_name} -o data *videos.zip* # Download the 512x512 videos

userTrain_videos.zip:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

submission_videos.zip:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

In [ ]:
!unzip -q data/submission_videos_resized_224.zip  -d {video_folder}
!unzip -q data/userTrain_videos_resized_224.zip -d {video_folder}

## Careful when running the below commands - For copying to Google Drive
# !rm data/submission_videos.zip data/userTrain_videos.zip 
# !cp -r data/ '/content/drive/MyDrive/mabe-2022-ant-beetle/data/'

# Train Unsupervised Baseline - SIMCLR 🏋️

- We use Contrastive learning for the baseline for the MABe video datasets. The code uses SIMCLR (A Simple Framework for Contrastive Learning of Visual Representations) - https://arxiv.org/abs/2002.05709 - A popular and "simple" contrastive learning algorithm.

- Some changes are made to SIMCLR use some specific ideas about the dataset. Namely frame stacking and cropping around the animal subjects.

- We use a ResNet50 model with the pytorch simclr package (unofficial) https://github.com/sthalles/SimCLR to do unsupervised learning on the video data. 

- We also stack past and future frames with frame skipping to incorporate temporal information into each of the embeddings.


## Dataloader 📚

The dataloader reads video files, seeks the required past and future frames with frame skipping.

Additionaly, we also crop the images to keep the animals in focus.

In [5]:
class AntBeetleVideoDataset(torch.utils.data.Dataset):
    """
    Reads frames from video files
    """
    def __init__(self, 
                 datafolder, 
                 frame_number_map,
                 keypoints,
                 frame_skip,
                 num_prev_frames, 
                 num_next_frames,
                 frame_size=(224, 224),
                 transform=None):
        """
        Initializing the dataset with images and labels
        """
        self.datafolder = datafolder
        self.transform = transform
        self.frame_number_map = frame_number_map
        self.num_prev_frames = num_prev_frames
        self.num_next_frames = num_next_frames
        self.frame_skip = frame_skip
        self.frame_size = frame_size
        self.keypoints = keypoints

        self._setup_frame_map()

    def set_transform(self, transform):
        self.transform = transform

    def _setup_frame_map(self):
        self._video_names = np.array(list(self.frame_number_map.keys()))
        # IMPORTANT: the frame number map should be sorted for self.get_video_name to work
        frame_nums = np.array([self.frame_number_map[k] for k in self._video_names])
        self._frame_numbers = frame_nums[:, 0] - 1 # start values
        assert np.all(np.diff(self._frame_numbers) > 0), "Frame number map is not sorted"

        self.length = frame_nums[-1, 1] # last value is the total number of frames

    def get_frame_info(self, global_index):
        """ Returns corresponding video name and frame number"""
        video_idx = np.searchsorted(self._frame_numbers, global_index) - 1
        frame_index = global_index - (self._frame_numbers[video_idx] + 1)
        return self._video_names[video_idx], frame_index
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        video_name, frame_index = self.get_frame_info(idx)

        video_path = os.path.join(self.datafolder, video_name + '.avi')
        
        nf = self.num_next_frames + self.num_prev_frames + 1
        frames_array = np.zeros((*self.frame_size, nf), dtype=np.float32)
        
        if not os.path.exists(video_path):
#             raise FileNotFoundError(video_path)
            if self.transform is not None:
                frames_array = self.transform(frames_array)
            return { "idx": idx,
                     "image": frames_array,
                    } 
        
        
        cap = cv2.VideoCapture(video_path)
        num_video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        for arridx, fnum in enumerate(range(frame_index - self.num_prev_frames * self.frame_skip,
                                            frame_index + self.num_next_frames * self.frame_skip + 1,
                                            self.frame_skip + 1)):
            if fnum < 0 or fnum >= num_video_frames:
                continue
            cap.set(cv2.CAP_PROP_POS_FRAMES, fnum)
            success, frame = cap.read()
            # print(fnum, frame_index, success)
            if success:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frames_array[:, :, arridx] = frame

        if video_name in self.keypoints['sequences']:
            bbox = self.keypoints['sequences'][video_name]['bbox']
            if bbox.shape[0] > frame_index:
                bbox = bbox[frame_index]
                frames_array = frames_array[bbox[0]:bbox[2], bbox[1]:bbox[3]] # Crop the image so random crop is more useful
        
        if self.transform is not None:
            frames_array = self.transform(frames_array)

        return { "idx": idx,
                 "image": frames_array,
               }             

## Utilites - Optimizer, Transforms and Augmentations 🔧

In [6]:
def load_optimizer(optimizer, epochs, weight_decay, batch_size, model):

    scheduler = None
    if optimizer == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)  # TODO: LARS
    elif optimizer == "LARS":
        # optimized using LARS with linear learning rate scaling
        # (i.e. LearningRate = 0.3 × BatchSize/256) and weight decay of 10−6.
        learning_rate = 0.3 * batch_size / 256
        optimizer = LARS(
            model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay,
            exclude_from_weight_decay=["batch_normalization", "bias"],
        )

        # "decay the learning rate with the cosine decay schedule without restarts"
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, epochs, eta_min=0, last_epoch=-1
        )
    else:
        raise NotImplementedError

    return optimizer, scheduler

def save_model(epoch, model_path, model, optimizer):
    out = os.path.join(model_path, "checkpoint_{}.tar".format(epoch))

    # To save a DataParallel model generically, save the model.module.state_dict().
    # This way, you have the flexibility to load the model any way you want to any device you want.
    if isinstance(model, torch.nn.DataParallel):
        torch.save(model.module.state_dict(), out)
    else:
        torch.save(model.state_dict(), out)

In [7]:
class TransformsSimCLR:
    def __init__(self, size, pretrained=True, n_channel=3, validation=False) -> None:
        self.train_transforms = T.Compose([
            T.ToTensor(),
            T.RandomResizedCrop(size=size, scale=(0.25, 1.0)),
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            # Taking the means of the normal distributions of the 3 channels
            # since we are moving to grayscale
            T.Normalize(mean=np.mean([0.485, 0.456, 0.406]).repeat(n_channel),
                        std=np.sqrt(
                            (np.array([0.229, 0.224, 0.225])**2).sum()/9).repeat(n_channel)
                        ) if pretrained is True else T.Lambda(lambda x: x)
        ])

        self.validation_transforms = T.Compose([
            T.ToTensor(),
            T.Resize(size=size),
            # Taking the means of the normal distributions of the 3 channels
            # since we are moving to grayscale
            T.Normalize(mean=np.mean([0.485, 0.456, 0.406]).repeat(n_channel),
                        std=np.sqrt(
                            (np.array([0.229, 0.224, 0.225])**2).sum()/9).repeat(n_channel)
                        ) if pretrained is True else T.Lambda(lambda x: x)
        ])
        
        self.validation = validation

    def __call__(self, x):
        if not self.validation:
            return self.train_transforms(x), self.train_transforms(x)
        else:
            return self.validation_transforms(x)

## Bounding box creation 📦 

Since most of the frame is empty, it is important that the ant beetle are cropped correctly when doing SimCLR augments. We use the keypoints to create rough bounding box coordinates around them.

Note that these bounding boxes are made in a simple fixed pixel size cropping mechanism, feel free to change the bounding box generation system.

In [8]:
######## Prepare bounding boxes from keypoints ##########

# Preparing some bounding box information to be used for cropping frames during training
keypoints = np.load(os.path.join(datafolder, 'submission_keypoints.npy'), allow_pickle=True).item()



# Can save it you want and load later
# keypoints = np.save(os.path.join(datafolder, 'submission_keypoints_bbox.npy'), keypoints)

# keypoints = np.load(os.path.join(datafolder, 'submission_keypoints_bbox.npy'), allow_pickle=True).item()

## Training ☑️ 

Below are hyperparamers you can play around with. The runs are pretty slow, so you can reduce the epochs and steps per epochs to find the parameters you want to use.

Note that we do not go over the entire dataset for each "epoch", because the whole dataset is huge. 

This code will only use the submission videos for unsupervised training, but you can change it to use all the videos.

In [9]:
################### CONFIG #########################
IS_PRETRAINED = True
batch_size = 32
epochs = 10

# Stack frames with frame skip from the video sequences
LEFT_WINDOW = 3
RIGHT_WINDOW = 3
IMG_SIZE = 224
FRAME_SKIP = 5
n_channel = LEFT_WINDOW + RIGHT_WINDOW + 1

# Check batch size that fits when changing this
embedding_size = 128

# Full Dataset is huge, he're we limiting to steps per epoch
steps_per_epoch = 1000

videos_folder = os.path.join(datafolder, 'video_clips') # TODO: Change this to combined folder
frame_number_map = np.load(os.path.join(datafolder, 'frame_number_map.npy'), allow_pickle=True).item()

checkpoint_folder = "antbeetle_checkpoints/" # Can change this to a Google drive folder
if not os.path.exists(checkpoint_folder): 
    os.mkdir(checkpoint_folder)

In [10]:
train_dataset = AntBeetleVideoDataset(datafolder=videos_folder, 
                      frame_number_map=frame_number_map,
                      keypoints=keypoints,
                      frame_skip=FRAME_SKIP, 
                      num_prev_frames=LEFT_WINDOW, 
                      num_next_frames=RIGHT_WINDOW,
                      frame_size=(224, 224),
                      transform=TransformsSimCLR(size=IMG_SIZE, 
                                                 pretrained=IS_PRETRAINED, 
                                                 n_channel=n_channel))


train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
    num_workers=2,
)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

##################### MODEL ############################
def get_simclr_model():
    resnet_encoder = torchvision.models.resnet50(pretrained=IS_PRETRAINED)

    ## Experimental setup for multiplying the grayscale channel
    ## https://stackoverflow.com/a/54777347
    weight = resnet_encoder.conv1.weight.clone()
    resnet_encoder.conv1 = torch.nn.Conv2d(n_channel, 64, kernel_size=7, stride=2, padding=3, bias=False)
    # normalize back by n_channels after tiling
    resnet_encoder.conv1.weight.data = weight.sum(dim=1, keepdim=True).tile(1, n_channel, 1, 1)/n_channel


    n_features = resnet_encoder.fc.in_features
    model = SimCLR(resnet_encoder, embedding_size, n_features)
    model = model.to(device)
    
    return model

model = get_simclr_model()

##################### UTILS ############################

optimizer_type = 'Adam'
weight_decay = 1e-6

optimizer, scheduler = load_optimizer(optimizer_type, epochs, weight_decay, batch_size, model)

world_size = 1
temperature = 0.5
criterion = NT_Xent(batch_size, temperature, world_size)

In [15]:
# Basic Training loop
def train(epoch, train_loader, model, criterion, optimizer):
    loss_epoch = 0
    
    # tqdm_iter = tqdm(train_loader, total=len(train_loader)) # Total train loader is huge, he're we limiting to steps per epoch
    tqdm_iter = tqdm(train_loader, total=steps_per_epoch)

    tqdm_iter.set_description(f"Epoch {epoch}")
    for step, batch in enumerate(tqdm_iter):
        optimizer.zero_grad()
        x_i = batch['image'][0].cuda(non_blocking=True)
        x_j = batch['image'][1].cuda(non_blocking=True)

        # positive pair, with encoding
        h_i, h_j, z_i, z_j = model(x_i, x_j)


        loss = criterion(z_i, z_j)
        loss.backward()

        optimizer.step()

        tqdm_iter.set_postfix(iter_loss=loss.item())

        loss_epoch += loss.item()
        if step >= steps_per_epoch:
            break

    return loss_epoch

In [16]:
# Baseline submission on the leaderboard is trained with 100 epochs, you can train according to your needs
for epoch in range(epochs):
    lr = optimizer.param_groups[0]['lr']
    loss_epoch = train(epoch, train_loader, model, criterion, optimizer)

    if scheduler:
        scheduler.step()

    if (epoch % 3) == 0:
        save_model(epoch, checkpoint_folder, model, optimizer)

save_model(epochs, checkpoint_folder, model, optimizer)

  0%|          | 0/1000 [00:01<?, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/home/sunchao/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


  0%|          | 0/1000 [00:00<?, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


  0%|          | 0/1000 [00:01<?, ?it/s]

...

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


  0%|          | 0/1000 [00:01<?, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb911988f70>
Traceback (most recent call last):
  File "/home/sunchao/anaconda3/envs/py38/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/sunchao/anaconda3/envs/py38/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/sunchao/anaconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb911988f70>
Traceback (most recent call last):
  File "/home/sun

  0%|          | 0/1000 [00:00<?, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


In [17]:
# Cleanup RAM
del model, optimizer
del train_loader, train_dataset

# Predict Embeddings 🔮

Here we'll predict the outputs from the frames, this may take a long time.

In [16]:
# Load latest model
model = get_simclr_model()
checkpoint_path = os.path.join(checkpoint_folder, 'checkpoint_10.tar')
model.load_state_dict(torch.load(checkpoint_path))
model = model.to(device)
model.eval();

In [17]:
prediction_dataset = AntBeetleVideoDataset(datafolder=videos_folder, 
                                          frame_number_map=frame_number_map,
                                          keypoints=keypoints,
                                          frame_skip=FRAME_SKIP,
                                          num_prev_frames=LEFT_WINDOW, 
                                          num_next_frames=RIGHT_WINDOW,
                                          frame_size=(224, 224),
                                          transform=TransformsSimCLR(size=IMG_SIZE, 
                                                                     pretrained=IS_PRETRAINED, 
                                                                     n_channel=n_channel,
                                                                     validation=True))

prediction_batch_size=128

prediction_loader = torch.utils.data.DataLoader(
    prediction_dataset,
    batch_size=prediction_batch_size,
    shuffle=False,
    drop_last=False,
    pin_memory=True,
    num_workers=2,
)

In [20]:
sample_submission = np.load(datafolder + 'sample_submission.npy')
submission = np.empty((sample_submission.shape[0], embedding_size), dtype=np.float32)
idx = 0
# This may take quite long, since predicting on all frames
for data in tqdm(prediction_loader, total=len(prediction_loader)):
    with torch.no_grad():
        images = data['image'].to(device)
        output = model.projector(model.encoder(images))
        output = output.cpu().numpy()
        submission[idx:idx+len(output)] = output
        idx += len(output)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


  0%|          | 0/66734 [00:00<?, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_38905/3383102623.py:9 in <cell line: 5>                                    │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_38905/3383102623.py'                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

# Submission 🚀

In [ ]:
print("Embedding shape:", submission.shape)

## Validate the submission ✅
The submssion should follow these constraints:

- It should be a numpy array
- Embeddings is an 2D numpy array of dtype float32
- The embedding size should't exceed 128
- The frame number map matches the clip lengths
- You can use the helper function below to check these

In [ ]:
def validate_submission(submission, frame_number_map):
    if not isinstance(submission, np.ndarray):
        print("Embeddings should be a numpy array")
        return False
    elif not len(submission.shape) == 2:
        print("Embeddings should be 2D array")
        return False
    elif not submission.shape[1] <= 128:
        print("Embeddings too large, max allowed is 128")
        return False
    elif not isinstance(submission[0, 0], np.float32):
        print(f"Embeddings are not float32")
        return False

    
    total_clip_length = frame_number_map[list(frame_number_map.keys())[-1]][1]
            
    if not len(submission) == total_clip_length:
        print(f"Emebddings length doesn't match submission clips total length")
        return False

    if not np.isfinite(submission).all():
        print(f"Emebddings contains NaN or infinity")
        return False

    print("All checks passed")
    return True

In [ ]:
validate_submission(submission, frame_number_map)

In [ ]:
np.save('submission_ant_simclr.npy', submission)

In [5]:
## Uploads may take time, you can also rund aicrowd-cli on your local machines with he prepared submission file
%aicrowd submission create --description "Ant-Beetle SimCLR Baseline" -c {aicrowd_challenge_name} -f submission_ant_simclr.npy

submission_ant_simclr.npy ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/4.4 GB • ? • -:--:--


Aborted!
